In [1]:
import os,shutil
import numpy as np
import pickle

class save_data(object):
    def __init__(self,data):
        self.data = data

with open("merge_data.pkl","rb") as f:
    data = pickle.load(f)
    
#dir, pktlen, payload len, flag, label
data_dir = {}

for item in data.data:

    label = item[0][-1]
    if label not in list(data_dir.keys()):
        data_dir[label] ={}
    
    for pkt in item:
        if pkt[0] == -1:
            continue
        lens = pkt[2] 
        if lens == 0:
            continue
        
        if lens not in list(data_dir[label].keys()):
            data_dir[label][lens] = 0
        data_dir[label][lens] += 1
        
data_dir_tuple = {}

for label in data_dir.keys():
    data_dir_tuple[label] = sorted(data_dir[label].items(),key=lambda item:item[1],reverse = True)
    
top20_dir = {}
for label in data_dir_tuple.keys():
    top20_dir[label] = sorted([item[0] for item in data_dir_tuple[label][:20]])
    
data_dir_adj = {}
for flow in data.data:
    label = flow[0][-1]
    
    if label not in list(data_dir_adj.keys()):
        data_dir_adj[label] = []
    tmp_flow = [item[1] for item in flow]
    adj_flow = []
    for item in tmp_flow:
        idx_val = [abs(item-cand) for cand in top20_dir[label]]
        adj_flow.append(top20_dir[label][np.argmin(np.array(idx_val))])
    data_dir_adj[label].append(adj_flow)


#define a trans seq of 20  top20_dir data_dir_adj
trans_dir_20 = {}
seq_len = 20
for label in data_dir_adj:
    if label not in list(trans_dir_20.keys()):
        trans_dir_20[label] = np.zeros((seq_len,20,20))
    top_20 = [item for item in top20_dir[label] ]
    #initial point
    for flow in data_dir_adj[label]:
#         for i in range(seq_len):
#             if i >len(flow)-1:
#                 break
        idx = top_20.index(flow[0])
        trans_dir_20[label][0][idx][idx] += 1
    
#     trans_dir_20[label][0] = trans_dir_20[label][0]/np.diagonal(trans_dir_20[label][0]).sum()
    #trans matrix
    
    for epoch in range(1,seq_len):
        for flow in data_dir_adj[label]:
            if epoch > len(flow)-1:
                continue
            idx_pre = top_20.index(flow[epoch-1])
            idx = top_20.index(flow[epoch])
            trans_dir_20[label][epoch][idx_pre][idx] += 1
        
        
#dir, pktlen, payload len, flag, label
data_len_dir = {}

for item in data.data:

    label = item[0][-1]
    if label not in list(data_len_dir.keys()):
        data_len_dir[label] =[]
    
    data_len_dir[label].append(item)

    
def randomSeq(maxValue, num):

    maxValue = int(maxValue)
    suiji_ser = random.sample(range(1,maxValue), k=num-1)
    suiji_ser.append(0)  
    suiji_ser.append(maxValue)
    suiji_ser = sorted(suiji_ser)
    per_all_persons = [ suiji_ser[i]-suiji_ser[i-1] for i in range(1, len(suiji_ser)) ] 
    
    return per_all_persons


In [2]:
#handshake seq counting
handshake_seq_dir = {}
for label in list(data_len_dir.keys()):
    tmp = {}
    for flow in data_len_dir[label]:
#         print(item)
        if flow[0][-3] not in list(tmp.keys()):
            tmp[flow[0][-3]] = 0
        tmp[flow[0][-3]] += 1
    
    tmp = sorted(tmp.items(), key=lambda d:d[1], reverse = True)
    handshake_seq_dir[label] = tmp[0][0]

In [ ]:
#untargeted attacking
import random

untargeted_flows_dir = {}
for src_clss in list(handshake_seq_dir.keys()):
    print("***** Refracting ", src_clss, " *****")

    can_flows = data_len_dir[src_clss]
    fund_len = 54
    
    adj_flows_untargetd = [handshake_seq_dir[src_clss][0],handshake_seq_dir[src_clss][2]]

#     org_flows_tmp = [flow for flow in can_flows]
#     org_flows = []
#     for item0 in org_flows_tmp:
#         org_flows.append([item[1] for item in item0])

    for flow in can_flows:
        tmp_flow = []
        
        payload_cul = 0

        for k in range(len(flow)):
            pkt = flow[k]
            #negative direction pkt
            if pkt[0] == -1:
                #without payload
                if pkt[2] == 0:
#                     tmp_flow.append(pkt[1])
                    continue
                #with payload
                if pkt[2] != 0:
#                     tmp_flow.append(pkt[1])
                    continue
            #positive direction pkt
            if pkt[0] ==1:
                #without payload
                if pkt[2] == 0:
                    tmp_flow.append(pkt[1])
                    continue
                #with payload
                if pkt[2] != 0:
                    payload = pkt[2]

                    if payload <=20:
                        tmp_flow.append(pkt[1])
                        continue
                    pkt_number = random.randint(1,int(payload/10))
                    seq = randomSeq(payload, pkt_number)
                    for i in range(len(seq)-1):
                        tmp_flow.append(fund_len+seq[i])
#                         tmp_flow.append(fund_len)

                    tmp_flow.append(fund_len+seq[-1])

                    continue

        adj_flows_untargetd.append(tmp_flow)
    untargeted_flows_dir[src_clss] = adj_flows_untargetd
                
        
        

In [ ]:
#Enhanced Untargeted attack, two items should be provided : trans_array => trans_dir_20, adjusted length(headers) => top20_dir
import random,math

untargeted_flows_dir = {}
for src_clss in list(handshake_seq_dir.keys()):
    print("***** Refracting ", src_clss, " *****")
    # ack_pad = False
    can_flows = data_len_dir[src_clss]


    payloads_blocks_list = []
    for flow in can_flows:
        #for each flow, first we gather the payload sequence blocks
        payloads_blocks = [] #(direction, start_idx, end_idx, flags(A,P), payload_len )
        i = 0
    #     print(flow)
        while i < len(flow)-1:
    #         print(flow[i])
    #     for i in range(len(flow)):
            if flow[i][2] == 0:
    #             payloads_blocks.append((flow[i][0],i,i,'A',0))
                i += 1
                continue
            start_idx = i
            payload_cul = []
            end_idx = 0
            #find the next opposite pkt
            for j in range(i,len(flow)):
                if flow[j][0] != flow[i][0] and flow[j][2] !=0:

                    end_idx = j
    #                 payload_cul = np.array([item[2] for item in flow[i:j+1]]).sum()
    #                 payloads_block.append(flow[i][0],start_idx,end_idx,payload_cul)
                    break
                end_idx = j
            payload_cul = np.array([item[2] for item in flow[i:j]]).sum()

    #         print(i,j)
            if end_idx == len(flow) -1:
                payloads_blocks.append((flow[i][0],start_idx,end_idx,payload_cul))
            else:
                payloads_blocks.append((flow[i][0],start_idx,end_idx-1,payload_cul))

            i = end_idx
        payloads_blocks_list.append(payloads_blocks)
    #     print(payloads_blocks)
    #         break
    #     break

    #payloads_blocks_list obtained
    fund_len = 54

    trans_flow_list = [] # pkt lens seq

    for k in range(len(can_flows)):
        flow = can_flows[k]
        hs_len = flow[0][-5]
        payloads_block = payloads_blocks_list[k]
        payload_cul = 0
        trans_idx = 0
        pre_lens_payload = 0
        trans_flow = []
        #append handshake pkts
        trans_flow.append(([66],0,0,'A'))
        trans_flow.append(([54],2,2,'A'))
    #     trans_flow.append(60)
    #     trans_flow.append((54,2,'A'))
    #     print(payloads_block)

    #     print(payloads_block)
        #select the initial point for the first block

        #get the first positive direction block
        first_idx = None
        for g in range(len(payloads_block)):
            if payloads_block[g][0] == 1:
                first_idx = g
                break
#             else:
#                 start_idx = payloads_block[g][1]
#                 end_idx = payloads_block[g][2]
#                 for kk in range(start_idx,end_idx+1):
#                     #append pos directional pkt only
#                     if flow[kk][0] == 1 and flow[kk][2] == 0:
#                         trans_flow.append(([flow[kk][1]],kk+hs_len,kk+hs_len,'A'))


        if first_idx == None: #error
            
            print("Error")
            print(payloads_block)
            continue

        for block in payloads_block[first_idx:]:
            if block[0] == -1:
#                 start_idx = block[1]
#                 end_idx = block[2]
#                 for kk in range(start_idx,end_idx+1):
#                     #append pos directional pkt only
#                     if flow[kk][0] == 1 and flow[kk][2] == 0:
#                         trans_flow.append(([flow[kk][1]],kk+hs_len,kk+hs_len,'A'))
                continue

            start_idx = block[1]
            end_idx = block[2]

            payload = block[-1]
            if payload == 0:
                continue

            #random seq
            if payload <=20:
                trans_flow.append(([payload + fund_len],start_idx+hs_len,end_idx+hs_len,'P'))
                continue
            pkt_number = random.randint(1,int(payload/10))
            seq = randomSeq(payload, pkt_number)
            seq = [item + fund_len for item in seq]
            seq.append(fund_len)
            
            trans_flow.append((seq,start_idx+hs_len,end_idx+hs_len,'P'))
        trans_flow_list.append((flow[0][-4], trans_flow))
    untargeted_flows_dir[src_clss] = trans_flow_list  
                    
    
        
        
        
        

In [8]:
import os,shutil
from scapy.all import *
from scapy.utils import PcapReader
import random
import copy

In [ ]:
# reverse to the original flows

root = 'data'
root_adj_undir = 'data_untargeted_undir'
root_adj_bidir = 'data_untargeted_bidir'
root_undir_org = 'data_flow_org'

if not os.path.exists(root_adj_undir):
    os.mkdir(root_adj_undir)
if not os.path.exists(root_undir_org):
    os.mkdir(root_undir_org)
if not os.path.exists(root_adj_bidir):
    os.mkdir(root_adj_bidir)


for label in list(untargeted_flows_dir.keys()):
    dir_path = os.path.join(root,label)
    dir_path_untargeted_undir = os.path.join(root_adj_undir,label)
    dir_path_untargeted_bidir = os.path.join(root_adj_bidir,label)
    dir_path_undir_org = os.path.join(root_undir_org,label)
    if not os.path.exists(dir_path_untargeted_undir):
        os.mkdir(dir_path_untargeted_undir)
    if not os.path.exists(dir_path_untargeted_bidir):
        os.mkdir(dir_path_untargeted_bidir)
    if not os.path.exists(dir_path_undir_org):
        os.mkdir(dir_path_undir_org)
    
    # flow level
    if label in ['SFTP','FTPS']:
        continue
    print("***** Reversing",label," *****")
    for flow_pair in untargeted_flows_dir[label]:
        file = flow_pair[0]
        blocks = flow_pair[1] #tuple ([86, 81, 152, 67], 13, 14, 'P')
        if os.path.exists(os.path.join(dir_path_untargeted_bidir,file)):
            continue
        try:
            
            
            #open file 
            org_flow = rdpcap(os.path.join(dir_path,file))
            
            #confirm the src/dst ip
            src_ip = None
            src_port = None
            src_mac = None
            dst_ip = None
            dst_port = None
            dst_mac = None
            for ff in org_flow:
                if ff['TCP'].flags == 'S':
                    src_ip = ff['IP'].src
                    dst_ip = ff['IP'].dst

                    src_port = ff['TCP'].sport
                    dst_port = ff['TCP'].dport

                    src_mac = ff['Ether'].src
                    dst_mac = ff['Ether'].dst     
                if ff['TCP'].flags == 'SA':
                    src_ip = ff['IP'].dst
                    dst_ip = ff['IP'].src

                    src_port = ff['TCP'].dport
                    dst_port = ff['TCP'].sport

                    src_mac = ff['Ether'].dst
                    dst_mac = ff['Ether'].src
                    break
            if src_ip == None:
                print('Error')
                print(file)
                break


            #idx reconstruction
            recon_blocks = []
            for i in range(len(blocks)-1):
                current_end_idx = blocks[i][-2]
                next_start_idx = blocks[i+1][1]

                recon_blocks.append((blocks[i],1))

                if current_end_idx + 1 != next_start_idx:
                    recon_blocks.append(((current_end_idx+1,next_start_idx-1),-1))
            recon_blocks.append((blocks[-1],1))
            # deal with the remaining pkts
            if blocks[-1][-2] < len(org_flow)-1:
                recon_blocks.append(((blocks[-1][-2]+1,len(org_flow)-1),-1))

            #traverse the recon_blocks
            # maintain a ack and a seq numbers
            idx = recon_blocks[0][1]
            ack = org_flow[idx]['TCP'].ack
            seq = org_flow[idx]['TCP'].seq

            # eastablish a standard pkt
            standard_pkt = None
            for pkt in org_flow:
                if len(pkt) >=70:
                    standard_pkt = pkt
            if standard_pkt == None:
                print("Standard pkt not found !")
                print(file)
                continue


            adj_flow_pcap_bidir = []
            adj_flow_pcap_undir = []

            for block_pair in recon_blocks:
                block = block_pair[0]

                #extract ack and seq number of the first pkt of each block
                idx = block[1]
                ack = org_flow[idx]['TCP'].ack
                seq = org_flow[idx]['TCP'].seq




                # positive direction blocks
                if block_pair[-1] == 1:
                    #[start,end)
                    start_idx = block[1]
                    end_idx = block[2] + 1

                    if block[-1] == 'A': #block without payload
                        for i in range(start_idx,end_idx):
                            tmp_pkt = copy.deepcopy(org_flow[i])
                            adj_flow_pcap_bidir.append(tmp_pkt)
                            adj_flow_pcap_undir.append(tmp_pkt)

                        continue
                    if block[-1] == 'P': #block with payload
                        # cumulate payload from one block
                        payload_cul = bytearray()
                        for i in range(start_idx,end_idx):
                            if org_flow[i].haslayer("Raw"):
                                payload_cul += bytearray(bytes(org_flow[i]['Raw']))
                        adj_seq = block[0]
                        #check out if the culmulate payload generated sequence equals to the original payload length
                        if np.array(adj_seq).sum() - np.array(adj_seq).shape[0]*54 != len(payload_cul):
                            print("Error! payloads length unequal!")
                            print(file)
                            break
                        # construct the fundamental pkt
                        pkt_recon_fund = copy.deepcopy(standard_pkt)
                        pkt_recon_fund["Ether"].src = src_mac
                        pkt_recon_fund["Ether"].dst = dst_mac
                        pkt_recon_fund["IP"].dst = dst_ip
                        pkt_recon_fund["IP"].src = src_ip
                        pkt_recon_fund["TCP"].dport = dst_port
                        pkt_recon_fund["TCP"].sport = src_port
                        # ack = ack[0]
                        pkt_recon_fund["TCP"].ack = ack

                        #random time seq
                        time_start = org_flow[block[1]].time
                        time_end = org_flow[block[2]].time
                        duration = time_end - time_start
    #                     print(duration)
                        time_seq = [duration/(len(adj_seq)*1.0) for x in range(len(adj_seq))]
                        if len(time_seq) > 3:

                            time_seq[1] += time_seq[0]
                            time_seq[0] = 0
                            time_seq[-2] += time_seq[-1]
                            time_seq[-1] = 0

                        # construct each pkt
                        payload_used_cul = 0
                        for i in range(len(adj_seq)):
                            gen_pkt_len = adj_seq[i]
                            gen_pkt_time = time_seq[i]
                            payload_used = gen_pkt_len - fund_len

                            # copy from fundamental pkt
                            pkt_recon = copy.deepcopy(pkt_recon_fund)
                            # seq[i] = seq[0] + payload_used_cul
                            pkt_recon["TCP"].seq = seq + payload_used_cul
                            pkt_recon.time = time_start + gen_pkt_time
                            time_start += gen_pkt_time
                            pkt_recon["TCP"].flags = "PA" #PSH ACK

                            # concat payload payload_cul
                            im_py = payload_cul[payload_used_cul:payload_used_cul+payload_used]
    #                         print(repr(pkt_recon))
                            pkt_recon['Raw'] = Packet(im_py)

                            payload_used_cul += payload_used

                            #convert pkt to btye flow
                            pkt_recon.len = len(pkt_recon)
                            pkt_recon['IP'].id += i
                            pkt_recon['IP'].len = (len(pkt_recon) -14)
    #                         print(len(pkt_recon))
                            #append to new flow
                            adj_flow_pcap_bidir.append(pkt_recon)
                            adj_flow_pcap_undir.append(pkt_recon)


                # negative direction blocks
                if block_pair[-1] == -1:
                    start_idx = block[0]
                    end_idx = block[1] + 1
                    for i in range(start_idx,end_idx):
                        adj_flow_pcap_bidir.append(org_flow[i])
                    #[start,end)
        except:
            print("Error")
            print(file)
            continue
        output_path_undir_org = os.path.join(dir_path_undir_org, file)
        output_path_untargeted_undir = os.path.join(dir_path_untargeted_undir,file)
        output_path_untargeted_bidir = os.path.join(dir_path_untargeted_bidir,file)
        
        # org_flow undirected
        org_flow_undir = []
        for pkt in org_flow:
            if pkt['IP'].src == src_ip:
                org_flow_undir.append(pkt)
        
        #write the org undir file
        if not os.path.exists(output_path_undir_org):
            
            writer = PcapWriter(output_path_undir_org)
            for pkt in org_flow_undir:
                writer.write(pkt)
            writer.close()
        
        #write the undirectional untargeted flow
        writer = PcapWriter(output_path_untargeted_undir)
        for pkt in adj_flow_pcap_undir:
            writer.write(pkt)
        writer.close()
        
        #write the bidirectional untargeted flow
        writer = PcapWriter(output_path_untargeted_bidir)
        for pkt in adj_flow_pcap_bidir:
            writer.write(pkt)
        writer.close()

                
                


            
            
        
        
        
        
#         org_undir_flow = []